# Labelspark - The Databricks and Labelbox connector

Thanks for trying out the Databricks and Labelbox Connector! Unless you downloaded this directly from our repository, you or someone from your organization signed up for a Labelbox trial through Databricks Partner Connect. This notebook helps illustrate how Labelbox and Databricks can be used together to power unstructured data workflows. 

Labelbox can be used to rapidly annotate a variety of unstructured data from your Data Lake ([images](https://labelbox.com/product/image), [video](https://labelbox.com/product/video), [text](https://labelbox.com/product/text), and [geospatial tiled imagery](https://docs.labelbox.com/docs/tiled-imagery-editor)) and the Labelbox Connector for Databricks makes it easy to bring the annotations back into your Lakehouse environment for AI/ML and analytical workflows.

<img src="https://labelbox.com/static/images/partnerships/collab-chart.svg" alt="example-workflow" width="800"/>

<b>Questions or comments? Reach out to us at [hello+databricks@labelbox.com](mailto:hello+databricks@labelbox.com)
  
Last Updated: December 20, 2022

# Set Up Labelspark Client

This tutorial notebook requires a Lablbox API Key. Please login to your [Labelbox Account](https://app.labelbox.com/) and generate an [API Key](https://docs.labelbox.com/reference/create-api-key)

In [0]:
%pip install labelspark --upgrade -q

In [0]:
import labelspark

In [0]:
api_key = ""

In [0]:
lbspk_client = labelspark.Client(api_key)

# **Create a Databricks Spark Table from a Labelbox Dataset**

`client.create_table_from_dataset()` will create a Databricks Spark table from a Labelbox dataset given the following: 
- `labelbox Client` object (`labelbox.client.Client`)
- An existing `labelbox_dataset` object `(labelbox.schema.dataset.Dataset)`
- An optional `metadata_index` dictionary
  - This must be a dictionary where {key=metadata_field_name : value=metadata_type} 
    - `metadata_field_name` will correspond to a column in your Databricks Spark table  
    - `metadata_type` must be `"string"`, `"enum"`, `"datetime"` or `"number"`

In [0]:
labelbox_dataset_id = ""
lb_dataset = lbspk_client.lb_client.get_dataset(labelbox_dataset_id)

metadata_index = {
    "split" : "enum"
}

In [0]:
spark_table = lbspk_client.create_table_from_dataset(
  lb_client=lbspk_client.lb_client, 
  lb_dataset=lb_dataset, 
  metadata_index=metadata_index
)

Labelbox data row export complete
 Start Time: 2022-12-20 06:03:05.308926
 End Time: 2022-12-20 06:03:05.660166
 Export Time: 0:00:00.351240
Data rows to create in table: 99
Success: Created table from Labelbox dataset with ID clbvre5170dj608036y5k80b0
 Start Time: 2022-12-20 06:03:05.660233
 End Time: 2022-12-20 06:03:05.799942
 Total Time: 0:00:00.139709
Data rows in table: 99

In [0]:
# Let's view your generated Databricks Spark table
spark_table.show(5)

+--------------------+--------------------+--------------------+--------------------+-----+
 data_row_id| global_key| external_id| row_data|split|
+--------------------+--------------------+--------------------+--------------------+-----+
clbvriadd1dm5074e...|2b319c4b-80f2-49f...|2b319c4b-80f2-49f...|https://storage.l...|valid|
clbvriadd1dm1074e...|3a6bf39a-cee6-465...|3a6bf39a-cee6-465...|https://storage.l...|valid|
clbvriadd1dlx074e...|97c5c991-99df-4c2...|97c5c991-99df-4c2...|https://storage.l...|valid|
clbvriadd1dlt074e...|10415ea4-a837-41e...|10415ea4-a837-41e...|https://storage.l...|valid|
clbvriadd1dlp074e...|c28ae2eb-0d96-4bc...|c28ae2eb-0d96-4bc...|https://storage.l...|valid|
+--------------------+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

# **Create Labelbox Data Rows from a Databricks Spark Table**

`client.create_data_rows_from_table()` will create Labelbox data rows given the following:
- `labelbox Client` object (`labelbox.client.Client`)
- An existing `labelbox_dataset` object `(labelbox.schema.dataset.Dataset)`
- An existing `spark_table` object
- A column name for your Labelbox data row `row_data`
- An optional column name for your Labelbox data row `global_key` - defaults to `row_data`
- An optional column name for your Labelbox data row `external_id` - defaults to `global_key`
- An optional `metadata_index` dictionary
  - This must be a dictionary where {key=metadata_field_name : value=metadata_type} 
    - `metadata_field_name` will correspond to a column in your Databricks Spark table  
    - `metadata_type` must be `"string"`, `"enum"`, `"datetime"` or `"number"`

In [0]:
# We're going to create a new_global_key column to use for uploading data
from pyspark.sql.functions import lit, concat
import uuid

new_suffix = f"_{str(uuid.uuid4())}"
spark_table = spark_table.withColumn('global_key_suffix', lit(new_suffix)) # Create a column that we can concatenate a fixed value on existing global keys
spark_table = spark_table.withColumn('new_global_key',concat(*['global_key', 'global_key_suffix'])) # Create a new_global_key column

# We're going to create a new dataset to upload data to, you can also pass in an existing dataset
lb_dataset = lbspk_client.lb_client.create_dataset(name="clone_dataset")
print(f"Labelbox Dataset: {lb_dataset.uid}")

metadata_index = {
    "split" : "enum"
}

spark_table.show(5)

Labelbox Dataset: clbvtkvn21mdw08zi5pau0dkx
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
 data_row_id| global_key| external_id| row_data|split| global_key_suffix| new_global_key|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
clbvriadd1dm5074e...|2b319c4b-80f2-49f...|2b319c4b-80f2-49f...|https://storage.l...|valid|_ea78a183-cd7d-40...|2b319c4b-80f2-49f...|
clbvriadd1dm1074e...|3a6bf39a-cee6-465...|3a6bf39a-cee6-465...|https://storage.l...|valid|_ea78a183-cd7d-40...|3a6bf39a-cee6-465...|
clbvriadd1dlx074e...|97c5c991-99df-4c2...|97c5c991-99df-4c2...|https://storage.l...|valid|_ea78a183-cd7d-40...|97c5c991-99df-4c2...|
clbvriadd1dlt074e...|10415ea4-a837-41e...|10415ea4-a837-41e...|https://storage.l...|valid|_ea78a183-cd7d-40...|10415ea4-a837-41e...|
clbvriadd1dlp074e...|c28ae2eb-0d96-4bc...|c28ae2eb-0d96-4bc...|https://storage.l...|valid|_ea78a183-cd7d-40...|c28ae2eb-0d96-4bc...|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
only showing top 5 rows

In [0]:
upload_results = lbspk_client.create_data_rows_from_table(
  lb_client=lbspk_client.lb_client, 
  spark_table=spark_table,
  lb_dataset=lb_dataset,
  row_data_col="row_data",
  global_key_col="new_global_key",
  metadata_index=metadata_index
)

Success: Labelbox upload conversion complete
 Start Time: 2022-12-20 06:03:07.640368
 End Time: 2022-12-20 06:03:07.745195
 Conversion Time: 0:00:00.104827
Data Rows to Upload: 99
Success: Uploaded table rows to Labelbox dataset with ID clbvtkvn21mdw08zi5pau0dkx
 Start Time: 2022-12-20 06:03:07.845029
 End Time: 2022-12-20 06:03:13.299734
 Upload Time: 0:00:05.454705
Data rows uploaded: 99

In [0]:
upload_results

Out[9]: []

# **Upsert Labelbox Metadata Values with Databricks Spark table column data**

client.upsert_labelbox_metadata() will upsert Labelbox metadata based on most recent column values from your Databricks Spark table given the following:

- `labelbox Client` object (`labelbox.client.Client`)
- An existing `labelbox_dataset` object `(labelbox.schema.dataset.Dataset)`
- A list of `global_keys` that correspond to data rows to-be-upserted
- A column name for your Labelbox data row `global_key`
- A `metadata_index` where each key is a metadata field you're looking to upsert in Labelbox
  - This must be a dictionary where {key=metadata field name : value=metadata type} 
    - `metadata_type` must be `"string"`, `"enum"`, `"datetime"` or `"number"`
  - Each key passed in to the `metadata index` will correspond to a column in your Databricks Spark table

In [0]:
# For this demo, let's change the existing metadata values in our Databricks Spark table and then upsert metadata values on Labelbox
spark_table = spark_table.withColumn("split", lit("train"))
spark_table.show(5)

+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
 data_row_id| global_key| external_id| row_data|split| global_key_suffix| new_global_key|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
clbvriadd1dm5074e...|2b319c4b-80f2-49f...|2b319c4b-80f2-49f...|https://storage.l...|train|_ea78a183-cd7d-40...|2b319c4b-80f2-49f...|
clbvriadd1dm1074e...|3a6bf39a-cee6-465...|3a6bf39a-cee6-465...|https://storage.l...|train|_ea78a183-cd7d-40...|3a6bf39a-cee6-465...|
clbvriadd1dlx074e...|97c5c991-99df-4c2...|97c5c991-99df-4c2...|https://storage.l...|train|_ea78a183-cd7d-40...|97c5c991-99df-4c2...|
clbvriadd1dlt074e...|10415ea4-a837-41e...|10415ea4-a837-41e...|https://storage.l...|train|_ea78a183-cd7d-40...|10415ea4-a837-41e...|
clbvriadd1dlp074e...|c28ae2eb-0d96-4bc...|c28ae2eb-0d96-4bc...|https://storage.l...|train|_ea78a183-cd7d-40...|c28ae2eb-0d96-4bc...|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
only showing top 5 rows

In [0]:
results = lbspk_client.upsert_labelbox_metadata(
  lb_client=lbspk_client.lb_client,
  spark_table=spark_table,
  global_key_col="new_global_key",
  global_keys_list = [], # Will take all global keys from table if False  
  metadata_index=metadata_index
)

Laeblbox metadata export complete
 Start Time: 2022-12-20 06:03:13.441418
 End Time: 2022-12-20 06:03:15.639768
 Export Time: 0:00:02.198350
Data rows in export: 99
Upsert Labelbox metadata complete
 Start Time: 2022-12-20 06:03:15.639826
 End Time: 2022-12-20 06:03:21.974107
 Total Time: 0:00:06.334281
Data rows upserted: 99

In [0]:
results # You can view metadata changes on Labelbox

Out[12]: []

# **Upsert Databricks Spark Table Values with the Labelbox Metadata**
`client.upsert_table_metadata()` will upsert a Databricks Spark table based on most recent metadata from Labelbox given the following:
- `labelbox Client` object (`labelbox.client.Client`)
- An existing `labelbox_dataset` object `(labelbox.schema.dataset.Dataset)`
- An existing `spark_table`
- A column name for your Labelbox data row `global_key`
- A `metadata_index` where each key is a column you're looking to update in your Databricks Spark table
  - This must be a dictionary where {key=metadata field name : value=metadata type} 
    - `metadata_type` must be `"string"`, `"enum"`, `"datetime"` or `"number"`
  - Each key passed in to the `metadata index` will correspond to a column in your Databricks Spark table

In [0]:
metadata_index = {
    "split" : "enum"
}

# We'll use the same spark_table, but instead we'll modify the metadata to the original values by referencing the original global_key column
spark_table.show(5) 

+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
 data_row_id| global_key| external_id| row_data|split| global_key_suffix| new_global_key|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
clbvriadd1dm5074e...|2b319c4b-80f2-49f...|2b319c4b-80f2-49f...|https://storage.l...|train|_ea78a183-cd7d-40...|2b319c4b-80f2-49f...|
clbvriadd1dm1074e...|3a6bf39a-cee6-465...|3a6bf39a-cee6-465...|https://storage.l...|train|_ea78a183-cd7d-40...|3a6bf39a-cee6-465...|
clbvriadd1dlx074e...|97c5c991-99df-4c2...|97c5c991-99df-4c2...|https://storage.l...|train|_ea78a183-cd7d-40...|97c5c991-99df-4c2...|
clbvriadd1dlt074e...|10415ea4-a837-41e...|10415ea4-a837-41e...|https://storage.l...|train|_ea78a183-cd7d-40...|10415ea4-a837-41e...|
clbvriadd1dlp074e...|c28ae2eb-0d96-4bc...|c28ae2eb-0d96-4bc...|https://storage.l...|train|_ea78a183-cd7d-40...|c28ae2eb-0d96-4bc...|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
only showing top 5 rows

In [0]:
spark_table = lbspk_client.upsert_table_metadata(
  lb_client=lbspk_client.lb_client, 
  spark_table=spark_table, 
  global_key_col="global_key", 
  metadata_index=metadata_index
)

Labelbox metadata export complete
 Start Time: 2022-12-20 06:03:22.146173
 End Time: 2022-12-20 06:03:23.303210
 Export Time: 0:00:01.157037
Data rows in export: 99
Upsert table metadata complete
 Start Time: 2022-12-20 06:03:23.303272
 End Time: 2022-12-20 06:03:23.345043
 Total Time: 0:00:00.041771
Data rows upserted: 99

In [0]:
spark_table.show(5)

+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
 data_row_id| global_key| external_id| row_data|split| global_key_suffix| new_global_key|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
clbvriadd1dm5074e...|2b319c4b-80f2-49f...|2b319c4b-80f2-49f...|https://storage.l...|valid|_ea78a183-cd7d-40...|2b319c4b-80f2-49f...|
clbvriadd1dm1074e...|3a6bf39a-cee6-465...|3a6bf39a-cee6-465...|https://storage.l...|valid|_ea78a183-cd7d-40...|3a6bf39a-cee6-465...|
clbvriadd1dlx074e...|97c5c991-99df-4c2...|97c5c991-99df-4c2...|https://storage.l...|valid|_ea78a183-cd7d-40...|97c5c991-99df-4c2...|
clbvriadd1dlt074e...|10415ea4-a837-41e...|10415ea4-a837-41e...|https://storage.l...|valid|_ea78a183-cd7d-40...|10415ea4-a837-41e...|
clbvriadd1dlp074e...|c28ae2eb-0d96-4bc...|c28ae2eb-0d96-4bc...|https://storage.l...|valid|_ea78a183-cd7d-40...|c28ae2eb-0d96-4bc...|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+
only showing top 5 rows